### Import Files

In [1]:
import tensorflow as tf
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, InputSpec
import tensorflow_addons as tfa

In [3]:
from tensorflow.keras.layers import BatchNormalization
BatchNormalization._USE_V2_BEHAVIOR = False

In [4]:
# Used to make the tensorflow_addons work (Used for Group Norm Operation)
!pip install typeguard

### Basic Layer Blocks

In [5]:
# RELU6 Layer
class Relu6(Layer):
    ''' ReLU6 Layer.
    
    Performs ReLU6 activation.
    '''
    
    def __init__(self):
        super(Relu6, self).__init__()
        self.relu6 = tf.nn.relu6
        
    @tf.function
    def call(self, inputs):
        return self.relu6(inputs)

In [6]:
# Batch Normalization Layer
class BatchNorm(Layer):
    ''' Batch Normalization Layer.
        
    Performs Batch Normalization.
    '''
    
    
    def __init__(self, scale=True, center=True):
        super(BatchNorm, self).__init__()        
        #self.bn = tf.keras.layers.BatchNormalization(scale=scale, center=center, trainable=True)
        self.bn = BatchNormalization(scale=scale, center=center, trainable=True)

    #@tf.function
    def call(self, inputs, training=True):
        return self.bn(inputs, training=training)

In [7]:
# 2D Convolution
class Convolution2D(Layer):
    '''Performs 2D Convolution without any activation.
    
    Used for 2D convolution including 1x1 convolution blocks.
    '''
    
    
    def __init__(self, filters, kernel_size, strides, padding):
        super(Convolution2D, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters = filters, kernel_size = kernel_size, 
                                            strides = strides, padding = padding)
        self.bn = BatchNorm()
        
    @tf.function
    def call(self, inputs):
        
        x = self.conv(inputs)
        x = self.bn(x)
        
        return x

In [8]:
# 2D Convolution, RELU6
class Convolution2D_RELU6(Layer):
    '''Performs 2D Convolution with RELU6 activation.
    
    2D Convolution with RELU6 activation.
    Used mainly for residual blocks in Mobilenet V2.
    '''
    
    
    def __init__(self, filters, kernel_size, strides, padding):
        super(Convolution2D_RELU6, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters = filters, kernel_size = kernel_size, 
                                            strides = strides, padding = padding)
        
        self.bn = BatchNorm()
        self.act = Relu6()
        
    @tf.function
    def call(self, inputs):
        
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.act(x)
        
        return x

In [9]:
# Average Pooling Layer
class AveragePooling(Layer):
    '''Average Pooling Layer.
    
    Used to perform Average pooling operation over the input tensors.
    '''
    
    
    def __init__(self, pool_size):
        super(AveragePooling, self).__init__()
        
        self.avgpool = tf.keras.layers.AveragePooling2D(pool_size=pool_size, padding="SAME")
        
    @tf.function
    def call(self, inputs):
        
        x = self.avgpool(inputs)
         
        return x

In [10]:
class DenseLayer(Layer):
    '''Dense Layer.
    
    Fully Connected Layer.
    '''
    
    
    def __init__(self, units):
        super(DenseLayer, self).__init__()
        
        self.dense = tf.keras.layers.Dense(units=units,
                                           kernel_initializer=tf.random_normal_initializer(stddev=0.01))
        
    @tf.function
    def call(self, inputs):
        
        x = self.dense(inputs)
        
        return x

In [11]:
class FlattenLayer(Layer):
    '''Flatten Layer.
    
    Used to flatten outputs after Convolutions.
    Dense Layer does not automatically manages the flatten.
    '''
    
    
    def __init__(self):
        super(FlattenLayer, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        
    @tf.function
    def call(self, inputs):
        
        x = self.flatten(inputs)
        
        return x

In [12]:
# Depthwise Convolution
class DepthwiseConvolution(Layer):
    ''' Depthwise Convolution Layer.
    
    Performs Depthwise Convolution with Batch Norm
    '''
    
    
    def __init__(self, kernel_size = 3, strides = 1, padding = "SAME"):
        super(DepthwiseConvolution, self).__init__()
        self.dconv = tf.keras.layers.DepthwiseConv2D(kernel_size, strides=strides,
                                     depth_multiplier=1,
                                     padding=padding)
        self.bn = BatchNorm()
    
    @tf.function
    def call(self, inputs):
        
        x = self.dconv(inputs)
        x = self.bn(x)
        
        return x

In [13]:
# Separable Convolution
class SeparableConvolution(Layer):
    ''' Separable Convolution Layer.
    
    Performs Separable Convolution.
    '''
    
    
    def __init__(self, filters = 32, kernel_size = 3, strides = 1, padding = "SAME", 
                 depth_multiplier = 1):
        super(SeparableConvolution, self).__init__()
        self.sconv = tf.keras.layers.SeparableConv2D(filters,kernel_size, strides=strides,
                                     depth_multiplier=depth_multiplier,
                                     padding=padding)
        self.bn = BatchNorm()
        self.act = Relu6()
    
    @tf.function
    def call(self, inputs):
        
        x = self.sconv(inputs)
        x = self.bn(x)
        x = self.act(x)
        
        return x 

In [14]:
# Group Normalization
class GroupNorm(Layer):
    ''' Group Normalization Layer.
    
    Divides the channels of your inputs into smaller sub groups 
    and normalizes these values based on their mean and variance.
    '''
    
    
    def __init__(self, groups=5, axis=3):
        super(GroupNorm, self).__init__()
        self.gnorm = tfa.layers.GroupNormalization(groups=groups, axis=axis)
    
    @tf.function
    def call(self, inputs):
        return self.gnorm(inputs)


In [15]:
# Layer to perform Residual Addition for Mobilenet V2
class AdditionLayer(Layer):
    ''' Addition Layer.
    
    Adds Output of Expansion block to inputs in case of Stride 1 Blocks.
    '''
    def __init__(self):
        super(AdditionLayer, self).__init__()
        self.add = tf.keras.layers.Add()
    
    @tf.function
    def call(self, input1, input2):
        return self.add([input1, input2])
    

In [16]:
# Global Average Pooling Layer
class GlobalAveragePooling(Layer):
    '''Global Average Pooling Layer.
    
    Used to perform Global Average pooling operation over the input tensors.
    '''
    
    
    def __init__(self):
        super(GlobalAveragePooling, self).__init__()
        
        self.gpool = tf.keras.layers.GlobalAveragePooling2D()
        
    @tf.function
    def call(self, inputs):
        
        x = self.gpool(inputs)
        
        return x


### Mobilenet V2 Object Detection API functions to help perform V2 operations

In [17]:
@tf.function
def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

In [18]:
@tf.function
def _split_divisible(num, num_ways, divisible_by=8):
    """Evenly splits num, num_ways so each piece is a multiple of divisible_by."""
    assert num % divisible_by == 0
    assert num / num_ways >= divisible_by
    # Note: want to round down, we adjust each split to match the total.
    base = num // num_ways // divisible_by * divisible_by
    result = []
    accumulated = 0
    for i in range(num_ways):
        r = base
        while accumulated + r < num * (i + 1) / num_ways:
          r += divisible_by
        result.append(r)
        accumulated += r
    assert accumulated == num
    return result

In [19]:
@tf.function
def _fixed_padding(inputs, kernel_size, rate=1):
    """Pads the input along the spatial dimensions independently of input size.

    Pads the input such that if it was used in a convolution with 'VALID' padding,
    the output would have the same dimensions as if the unpadded input was used
    in a convolution with 'SAME' padding.

    Args:
    inputs: A tensor of size [batch, height_in, width_in, channels].
    kernel_size: The kernel to be used in the conv2d or max_pool2d operation.
    rate: An integer, rate for atrous convolution.

    Returns:
    output: A tensor of size [batch, height_out, width_out, channels] with the
      input, either intact (if kernel_size == 1) or padded (if kernel_size > 1).
    """
    kernel_size_effective = [kernel_size[0] + (kernel_size[0] - 1) * (rate - 1),
                           kernel_size[0] + (kernel_size[0] - 1) * (rate - 1)]
    pad_total = [kernel_size_effective[0] - 1, kernel_size_effective[1] - 1]
    pad_beg = [pad_total[0] // 2, pad_total[1] // 2]
    pad_end = [pad_total[0] - pad_beg[0], pad_total[1] - pad_beg[1]]
    padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg[0], pad_end[0]],
                                  [pad_beg[1], pad_end[1]], [0, 0]])
    return padded_inputs

In [20]:
@tf.function
def expand_input_by_factor(n, divisible_by=8):
    return lambda num_inputs, **_: _make_divisible(num_inputs * n, divisible_by)

### Complex Layer Blocks

In [21]:
class ExpandedConvolutionStride1(Layer):
    ''' Expanded Convolution Layer.
        
    Used for Residual blocks of Mobilenet V2 with Stride 1 Blocks.
    Input -> Expansion Block + Input -> Output.
    '''
    
    
    def __init__(self, input_filters, filters, kernel, block_stride=1, padding="SAME", expansion_factor=6):
        super(ExpandedConvolutionStride1, self).__init__()
        
        self.conv1 = Convolution2D_RELU6(input_filters*expansion_factor, (1, 1), 1, padding)
        self.dconv1 = DepthwiseConvolution(strides=block_stride)
        self.conv2 = Convolution2D(filters, (1, 1), 1, padding)
        self.add = AdditionLayer()

    @tf.function
    def call(self, inputs, training = True):
        
        x = self.conv1(inputs)
        x = self.dconv1(x)
        x = self.conv2(x)
        x = self.add(x, inputs)
        
        return x

In [22]:
class ExpandedConvolutionStride2(Layer):
    ''' Expanded Convolution Layer.
        
    Used for Residual blocks of Mobilenet V2 with Stride 2 Blocks.
    Input -> Expansion Block -> Output.
    '''
    
    
    def __init__(self, input_filters, filters, kernel, block_stride=2, padding="SAME", expansion_factor=6):
        super(ExpandedConvolutionStride2, self).__init__()        
        
        self.conv1 = Convolution2D_RELU6(input_filters*expansion_factor, (1, 1), 1, padding)
        self.dconv1 = DepthwiseConvolution(strides=block_stride)
        self.conv2 = Convolution2D(filters, (1, 1), 1, padding)

    @tf.function
    def call(self, inputs, training = True):
        x = self.conv1(inputs)
        x = self.dconv1(x)
        x = self.conv2(x)
        
        return x

In [23]:
class ExpandedConvolution(Layer):
    ''' Expanded Convolution Layer.
        
    Used for Residual blocks of Mobilenet V2 with Stride 1 Blocks.
    Input -> Expansion Block -> Output.
    '''
    
    
    def __init__(self, input_filters, filters, kernel, block_stride=1, padding="SAME", expansion_factor=6):
        super(ExpandedConvolution, self).__init__()        
        
        self.dconv1 = DepthwiseConvolution(strides=block_stride)
        self.conv2 = Convolution2D(filters, (1, 1), block_stride, padding)
        #self.add = AdditionLayer()

    @tf.function
    def call(self, inputs, training = True):
        
        x = self.dconv1(inputs)
        x = self.conv2(x)
        #x = self.add(x, inputs)
        
        return x

In [24]:
class ExpandedConvolutionDiff(Layer):
    ''' Expanded Convolution Layer Diff.
        
    Used for Residual blocks of Mobilenet V2 with Stride 1 blocks with different channels.
    Used for other than first bottleneck layer.
    Input -> Expansion Block -> Output.
    '''
    
    
    def __init__(self, input_filters, filters, kernel, block_stride=1, padding="SAME", expansion_factor=6):
        super(ExpandedConvolutionDiff, self).__init__()        
        
        self.conv1 = Convolution2D_RELU6(input_filters*expansion_factor, (1, 1), 1, padding)
        self.dconv1 = DepthwiseConvolution(strides=block_stride)
        self.conv2 = Convolution2D(filters, (1, 1), block_stride, padding)
        #self.add = AdditionLayer()

    @tf.function
    def call(self, inputs, training = True):
        
        x = self.conv1(inputs)
        x = self.dconv1(x)
        x = self.conv2(x)
        #x = self.add(x, inputs)
        
        return x

### Mobilenet V2 Block

In [25]:
class MobilenetV2(Model):
    ''' Mobilenet V2.
        Mobilenet V2 Layer Architecture.
    '''
    
    def __init__(self, num_outputs):
        super(MobilenetV2, self).__init__()
        
        # Layer - 1, Convolution 2D, 32 Output Channels, "SAME" padding
        self.conv1 = Convolution2D(32, (3, 3), (2, 2), "SAME")
        
        # Layer - 2, Inverted Residuals and Linear Bottlenecks
        self.exp1 = ExpandedConvolution(input_filters=32, filters=16, kernel = (3, 3), # Input Channels - 32
                                               expansion_factor=1) # Output Channels 16, stride = 1
        
        # Layer - 3, Inverted Residuals and Linear Bottlenecks
        self.exp2 = ExpandedConvolutionStride2(input_filters=16, filters=24, kernel = (3, 3), # Input Channels - 16
                                               expansion_factor=6) # Output Channels 24, stride = 2
        
        # Layer - 4, Inverted Residuals and Linear Bottlenecks
        self.exp3 = ExpandedConvolutionStride1(input_filters=24, filters=24, kernel = (3, 3), # Input Channels - 24
                                               expansion_factor=6) # Output Channels 24, stride = 1
        
        # Layer - 5, Inverted Residuals and Linear Bottlenecks
        self.exp4 = ExpandedConvolutionStride2(input_filters=24, filters=32, kernel = (3, 3), # Input Channels - 24
                                               expansion_factor=6) # Output Channels 32, stride = 2
        
        # Layer - 6, Inverted Residuals and Linear Bottlenecks
        self.exp5 = ExpandedConvolutionStride1(input_filters=32, filters=32, kernel = (3, 3), # Input Channels - 32
                                               expansion_factor=6) # Output Channels 32, stride = 1
        
        # Layer - 7, Inverted Residuals and Linear Bottlenecks
        self.exp6 = ExpandedConvolutionStride1(input_filters=32, filters=32, kernel = (3, 3), # Input Channels - 32
                                               expansion_factor=6) # Output Channels 32, stride = 1
        
        # Layer - 8, Inverted Residuals and Linear Bottlenecks
        self.exp7 = ExpandedConvolutionStride2(input_filters=32, filters=64, kernel = (3, 3), # Input Channels - 32
                                               expansion_factor=6) # Output Channels 64, stride = 2
        
        # Layer - 9, Inverted Residuals and Linear Bottlenecks
        self.exp8 = ExpandedConvolutionStride1(input_filters=64, filters=64, kernel = (3, 3), # Input Channels - 64
                                               expansion_factor=6) # Output Channels 64, stride = 1
        # Layer - 10, Inverted Residuals and Linear Bottlenecks
        self.exp9 = ExpandedConvolutionStride1(input_filters=64, filters=64, kernel = (3, 3), # Input Channels - 64
                                               expansion_factor=6) # Output Channels 64, stride = 1
        
        # Layer - 11, Inverted Residuals and Linear Bottlenecks
        self.exp10 = ExpandedConvolutionStride1(input_filters=64, filters=64, kernel = (3, 3), # Input Channels - 64
                                               expansion_factor=6) # Output Channels 48, stride = 1
        
        # Layer - 12, Inverted Residuals and Linear Bottlenecks
        self.exp11 = ExpandedConvolutionDiff(input_filters=64, filters=96, kernel = (3, 3), # Input Channels - 64
                                               expansion_factor=6) # Output Channels 96, stride = 1
        
        # Layer - 13, Inverted Residuals and Linear Bottlenecks
        self.exp12 = ExpandedConvolutionStride1(input_filters=96, filters=96, kernel = (3, 3), # Input Channels - 96
                                               expansion_factor=6) # Output Channels 64, stride = 1
        
        # Layer - 14, Inverted Residuals and Linear Bottlenecks
        self.exp13 = ExpandedConvolutionStride1(input_filters=96, filters=96, kernel = (3, 3), # Input Channels - 96
                                               expansion_factor=6) # Output Channels 96, stride = 1
        
        # Layer - 15, Inverted Residuals and Linear Bottlenecks
        self.exp14 = ExpandedConvolutionStride2(input_filters=96, filters=160, kernel = (3, 3), # Input Channels - 96
                                               expansion_factor=6) # Output Channels 160, stride = 2
        
        # Layer - 16, Inverted Residuals and Linear Bottlenecks
        self.exp15 = ExpandedConvolutionStride1(input_filters=160, filters=160, kernel = (3, 3), # Input Channels - 160
                                               expansion_factor=6) # Output Channels 160, stride = 1
        
        # Layer - 17, Inverted Residuals and Linear Bottlenecks
        self.exp16 = ExpandedConvolutionStride1(input_filters=160, filters=160, kernel = (3, 3), # Input Channels - 160
                                               expansion_factor=6) # Output Channels 160, stride = 1
        
        # Layer - 18, Inverted Residuals and Linear Bottlenecks
        self.exp17 = ExpandedConvolutionDiff(input_filters=160, filters=320, kernel = (3, 3), # Input Channels - 160
                                               expansion_factor=6) # Output Channels 320, stride = 1
        
        
        # Layer - 19, Inverted Residuals and Linear Bottlenecks
        self.conv2 = Convolution2D(1280, (1, 1), (1, 1), "SAME")
        
        # Layer - 20, Average Pool Layer
        self.avgpool = AveragePooling((7, 7))
        
        # Layer - 21, Inverted Residuals and Linear Bottlenecks
        self.conv3 = Convolution2D(num_outputs, (1, 1), (1, 1), "SAME")
        
        # Flatten Layer
        self.flat = FlattenLayer()
        
    def build(self, input_shape):
        super(MobilenetV2, self).build(input_shape)
        
    @tf.function
    def call(self, inputs):
        
        # Layer - 1, 2D Conv - Channels (3 -> 32)
        x = self.conv1(inputs)
        print("Shape 0 check")
        print(x.shape)
        
        x = self.exp1(x)
        print("Shape 1 check")
        print(x.shape)
        x = self.exp2(x)
        print("Shape 2 check")
        print(x.shape)
        x = self.exp3(x)
        print("Shape 3 check")
        print(x.shape)
        x = self.exp4(x)
        print("Shape 4 check")
        print(x.shape)
        x = self.exp5(x)
        print("Shape 5 check")
        print(x.shape)
        x = self.exp6(x)
        print("Shape 6 check")
        print(x.shape)
        x = self.exp7(x)
        print("Shape 7 check")
        print(x.shape)
        x = self.exp8(x)
        print("Shape 8 check")
        print(x.shape)
        x = self.exp9(x)
        print("Shape 9 check")
        print(x.shape)
        x = self.exp10(x)
        print("Shape 10 check")
        print(x.shape)
        x = self.exp11(x)
        x = self.exp12(x)
        x = self.exp13(x)
        x = self.exp14(x)
        x = self.exp15(x)
        x = self.exp16(x)
        x = self.exp17(x)
        
        x = self.conv2(x)
        print("Check before Avg pool")
        print(x.shape)
        x =  self.avgpool(x)
        print("Avg pool check")
        print(x.shape)
        x = self.conv3(x)
        x = self.flat(x)
        print("Flatten Check")
        print(x.shape)
        
        return x

### MobilenetV2 Details

In [26]:
# Dummy Data to set the inputs
s = (20, 224, 224, 3)
nx = np.random.rand(*s).astype(np.float32)/ 255
print(nx.shape)

(20, 224, 224, 3)


In [27]:
# MobilenetV2 Model Object
num_outputs = 1000 # Output Channels
m2 = MobilenetV2(num_outputs)

In [28]:
# Setting input shape for the model
# Setting input shapes manually, as we are not calling model.fit
m2._set_inputs(nx)

Shape 0 check
(None, 112, 112, 32)
Shape 1 check
(None, 112, 112, 16)
Shape 2 check
(None, 56, 56, 24)
Shape 3 check
(None, 56, 56, 24)
Shape 4 check
(None, 28, 28, 32)
Shape 5 check
(None, 28, 28, 32)
Shape 6 check
(None, 28, 28, 32)
Shape 7 check
(None, 14, 14, 64)
Shape 8 check
(None, 14, 14, 64)
Shape 9 check
(None, 14, 14, 64)
Shape 10 check
(None, 14, 14, 64)
Check before Avg pool
(None, 7, 7, 1280)
Avg pool check
(None, 1, 1, 1280)
Flatten Check
(None, 1000)


In [29]:
# Training Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(nx)
train_dataset = train_dataset.shuffle(buffer_size=10).batch(5)

In [30]:
# Sample Outputs
indices = [0, 1, 2, 3, 4]
depth = num_outputs
sample_labels = tf.one_hot(indices, depth)
print(sample_labels.shape)

(5, 1000)


In [31]:
# Model Structure Summary
m2.summary()

Model: "mobilenet_v2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolution2d (Convolution2D multiple                  1024      
_________________________________________________________________
expanded_convolution (Expand multiple                  1040      
_________________________________________________________________
expanded_convolution_stride2 multiple                  5784      
_________________________________________________________________
expanded_convolution_stride1 multiple                  9768      
_________________________________________________________________
expanded_convolution_stride2 multiple                  10960     
_________________________________________________________________
expanded_convolution_stride1 multiple                  16096     
_________________________________________________________________
expanded_convolution_stride1 multiple                 

In [33]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)

In [34]:
epochs = 2
loss_metric = tf.keras.metrics.Mean()

# Iterate over epochs.
for epoch in range(epochs):
    print('Start of epoch %d' % (epoch,))

    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            x = m2(x_batch_train)
            print(x.shape)
            
            
            # Compute reconstruction loss
            loss = tf.nn.softmax_cross_entropy_with_logits(logits=x, labels=sample_labels)
            print(loss)

        grads = tape.gradient(loss, m2.trainable_weights)
        optimizer.apply_gradients(zip(grads, m2.trainable_weights))

        loss_metric(loss)

        if step % 100 == 0:
            print('step %s: mean loss = %s' % (step, loss_metric.result()))
            
# NOTE: tf2test/MobilenetV2 is directory path, "checkpoint" at the end is for the name of checkpoint
m2.save_weights('../../tf2test/MobilenetV2/MobilenetV2')

Start of epoch 0
(5, 1000)
tf.Tensor([7.1081867 7.0731325 7.063461  8.24783   6.6261244], shape=(5,), dtype=float32)
step 0: mean loss = tf.Tensor(7.2237473, shape=(), dtype=float32)
(5, 1000)
tf.Tensor([8.023547  6.384165  7.3467064 6.719557  7.3176236], shape=(5,), dtype=float32)
(5, 1000)
tf.Tensor([8.023454  7.9393263 7.6722393 6.3961515 8.526684 ], shape=(5,), dtype=float32)
(5, 1000)
tf.Tensor([6.3303614 6.551711  7.300639  9.483231  6.7820773], shape=(5,), dtype=float32)
Start of epoch 1
(5, 1000)
tf.Tensor([6.531643  8.065785  6.307708  8.374764  6.1291857], shape=(5,), dtype=float32)
step 0: mean loss = tf.Tensor(7.293012, shape=(), dtype=float32)
(5, 1000)
tf.Tensor([6.7390175 7.64649   6.1010647 6.481074  6.662692 ], shape=(5,), dtype=float32)
(5, 1000)
tf.Tensor([5.9124413 6.081244  7.5194645 5.9521675 7.5330677], shape=(5,), dtype=float32)
(5, 1000)
tf.Tensor([6.435672  5.931156  8.289909  7.5436277 6.839227 ], shape=(5,), dtype=float32)


### SSD Lite